In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import resnet, densenet, vgg16, vgg19
from tensorflow.keras.applications.resnet import preprocess_input as resnet_preprocess_input
from tensorflow.keras.applications.densenet import preprocess_input as densenet_preprocess_input
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg_preprocess_input

# Load pre-trained ResNet50 model
resnet_model = resnet.ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Load pre-trained DenseNet121 model
densenet_model = densenet.DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Load pre-trained VGG16 model
vgg_model = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))



58889256/58889256 [==============================] - 6s 0us/step


In [21]:
train_dataset = []
# Preprocess input images for each model
def preprocess_image(img_path, model_name):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    if model_name == 'resnet':
        img_array = resnet_preprocess_input(img_array)
    elif model_name == 'densenet':
        img_array = densenet_preprocess_input(img_array)
    elif model_name == 'vgg':
        img_array = vgg_preprocess_input(img_array)
    img_array = tf.expand_dims(img_array, 0)  # Add batch dimension
    return img_array

# Example usage
for i in range(10):
    img_path = 'D:/Dersler-20210512T064119Z-001/Dersler/2023_Fall/COMP411/COMP411-FineART/toy_dataset/toy_dataset/{}.jpg'.format(i+1)
    resnet_input = preprocess_image(img_path, 'resnet')
    resnet_features = resnet_model.predict(resnet_input)
    train_dataset.append(resnet_features)

    
densenet_input = preprocess_image(img_path, 'densenet')
vgg_input = preprocess_image(img_path, 'vgg')

# Extract features from pre-trained models
resnet_features = resnet_model.predict(resnet_input)
densenet_features = densenet_model.predict(densenet_input)
vgg_features = vgg_model.predict(vgg_input)



1/1 [==============================] - 0s 246ms/step


In [6]:
print(resnet_features)

[[[[0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    1.4155909 ]
   [0.         0.         0.         ... 0.         0.
    1.6893629 ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.01841718
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]

  [[0.         0.06271434 0.         ... 0.         0.
    0.25866926]
   [0.         0.0669893  0.         ... 0.         0.
    2.0966072 ]
   [0.         0.         0.         ... 0.         0.
    0.48695505]
   ...
   [0.         0.         0.         ... 0.         2.1098237
    0.        ]
   [0.         0.         0.         ... 0.03330207 1.3731451
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]

  [[0.         0.         0.         ... 0.         0.23826277
    0.        ]
   [0.         0.24416709 0. 

In [15]:
base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
print(len(base_model.layers))

import numpy as np
import pandas as pd
import random
import os
import csv

toy_data_path = 'toy_dataset_label.csv'
toy_data = pd.read_csv(toy_data_path, on_bad_lines='skip', delimiter='\t',quoting=csv.QUOTE_NONE)


175


In [27]:
labels_toy_data = toy_data.iloc[:, 9].values[:10]

a = set()
for val in labels_toy_data:
    if pd.notna(val):
        a.add(val)
num_classes = len(a)

"""# Freeze some layers for fine-tuning
for layer in base_model.layers[:170]:
    layer.trainable = False

# Add new layers for your specific task
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(num_classes, activation='softmax')  # Adjust num_classes based on your task
])


# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
y= []
for elm in labels_toy_data:
    y.append(np.array([elm]))
print(np.array(y))
model.fit(x=train_dataset, y=y, epochs=5)"""

#########################################
import logging
import torch
import torch.nn as nn
import torchvision
from torchvision import models
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from collections import Counter

import numpy as np

data_aug_transforms = transforms.Compose([transforms.RandomCrop(500, pad_if_needed=True), transforms.ToTensor()])

def load_dataset():
    data_path = '/src/wikiart'
    #data_path = '/content/wikitest'
    dataset = torchvision.datasets.ImageFolder(
        root=data_path,
        transform=data_aug_transforms
    )
    #train_mask = list(range(num_training))
    length = len(dataset)
    num_training= int(0.6 * length)
    num_validation = int(0.2 * length)
    num_test = length - num_training - num_validation
    lengths = [num_training, num_validation, num_test] 

    train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, lengths)
    #print(len(train_dataset))

    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=batch_size,
        num_workers=0,
        shuffle=True
    )
    #val_mask = list(range(num_training, num_training + num_validation))
    #val_dataset = torch.utils.data.Subset(dataset, mask)
    val_loader = torch.utils.data.DataLoader(
        val_dataset,
        batch_size=batch_size,
        num_workers=0,
        shuffle=True
    )

    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=batch_size,
        num_workers=0,
        shuffle=True
    )
    return train_loader, val_loader, test_loader, test_dataset

[['mythological']
 ['mythological']
 ['genre']
 ['portrait']
 ['portrait']
 ['landscape']
 ['landscape']
 ['religious']
 ['other']
 ['other']]
Epoch 1/5


ValueError: in user code:

    File "C:\Users\busr4\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\busr4\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\busr4\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\busr4\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\busr4\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\busr4

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm

# Assuming you have a dataset class, let's call it CustomDataset, with images and labels

# Define transforms for training data
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Create your custom dataset instance
dataset = CustomDataset(root='path/to/your/images', transform=transform)

# Split the dataset into training and testing sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_set, test_set = random_split(dataset, [train_size, test_size])

# Create data loaders
train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
test_loader = DataLoader(test_set, batch_size=64, shuffle=False)

# Load the pre-trained ResNet model
resnet_model = models.resnet18(pretrained=True)
# Change the output layer to match the number of classes in your dataset
num_classes = len(dataset.classes)
resnet_model.fc = nn.Linear(resnet_model.fc.in_features, num_classes)

# Set up loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet_model.parameters(), lr=0.001, momentum=0.9)

# Train the model
num_epochs = 5
for epoch in range(num_epochs):
    resnet_model.train()
    for inputs, labels in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}'):
        optimizer.zero_grad()
        outputs = resnet_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

# Evaluate the model on the test set
resnet_model.eval()
all_predictions = []
all_labels = []
with torch.no_grad():
    for inputs, labels in tqdm(test_loader, desc='Testing'):
        outputs = resnet_model(inputs)
        _, predictions = torch.max(outputs, 1)
        all_predictions.extend(predictions.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Now, 'all_predictions' contains the predicted labels for the test set

# You can use the trained model to predict labels for the rest of your images in a similar way